In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述

    # グラフのインスタンスを作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=model_name)

    # ツールの定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    # ツール定義の紐づけ
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノードの作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", chatbot)

    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付エッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition, # ツール呼出と判断したらツールノードを呼ぶ
    )

    # ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
    # ツールからチャットボットへの戻りエッジを作成
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    # 記憶を持つ実行可能なステートグラフの作成
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)

    return graph


# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        if ("finish_reason" in event["messages"][-1].response_metadata and 
            event["messages"][-1].response_metadata["finish_reason"] == "stop"):
            print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述

graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述

# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    print()
    print(user_input)
    stream_graph_updates(graph, user_input)



こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？

1たす2は？
1たす2は3です。何か他に知りたいことがありますか？

台湾観光について検索結果を教えて
台湾観光に関する情報をいくつかご紹介します。

1. [阪急交通社の台湾観光ガイド](https://www.hankyu-travel.com/guide/taiwan/)
   - 台湾は絶景、グルメ、ショッピング、温泉などが楽しめる観光地で、台北のシンボルタワー「台北101」や、夜市でのB級グルメ、九份の街歩きが特におすすめです。
   - 人気の観光スポットには、九份、士林夜市、日月潭、野柳、龍虎塔、太魯閣溪谷などがあり、それぞれの魅力を楽しむことができます。

2. [近畿日本ツーリストの台北観光スポット19選](https://www.knt.co.jp/travelguide/kaigai/027/)
   - 台湾は日本から直行便で約4時間の距離にあり、手軽に訪れることができます。台北には国立故宮博物院や中正紀念堂、士林観光夜市など、歴史と文化を感じられるスポットが多数あります。
   - また、ノスタルジックな雰囲気の九份や、美味しいグルメを楽しめる夜市も人気です。

台湾には多様な観光スポットがあり、食事やショッピング、歴史的な観光地を楽しむことができます。興味のある場所についてさらに詳しく知りたい場合は、お知らせください！
ありがとうございました!
